In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
from scipy import stats
import statsmodels.api as sm
from scipy.stats import ks_2samp
import scipy.stats as stats
from scipy.stats import ttest_rel

In [2]:
fpcomdb_data = pd.read_excel(r'20200628_FP.com database.xlsx')
fpcomdb_data.to_pickle('fpcomdb_data.pkl')
esso_data =  pd.read_excel(r'Esso Report YTD v3 - wk17.xlsx',sheetname='DATA')
esso_data.to_pickle('esso_data.pkl')

In [11]:
fpcomdb_data.columns = fpcomdb_data.columns.str.strip()
fpcomdb_data.columns = fpcomdb_data.columns.str.replace(" ","")
esso_data.columns = esso_data.columns.str.replace(" ","")

In [60]:
fpcomdb_data_res = fpcomdb_data.set_index(['PBL', 'Site.ReportingSite',\
                                           'SiteName', 'Date','Product']).unstack('Product').reset_index()
fpcomdb_data_res.columns = [''.join(col).strip() for col in fpcomdb_data_res.columns.values]
fpcomdb_data_res.columns = fpcomdb_data_res.columns.str.replace(" ","")

In [65]:
mapfile = pd.read_excel(r'Site ID vs PBL.xlsx')
esso_data = esso_data.merge(mapfile[['Site ID','PBL']],how= 'left',left_on='SiteNumber',right_on='Site ID')
esso_data.isnull().sum()
esso_data.shape
esso_data.loc[esso_data.PBL.isnull(),['SiteNumber','SiteName']].drop_duplicates()

In [90]:
esso_data.to_pickle('esso_data.pkl')
fpcomdb_data_res.to_pickle('fpcomdb_data_res.pkl')

In [81]:
fpcomdb_data_res_todate = fpcomdb_data_res.query('Date >= "2020-03-08"')
esso_data_todate = esso_data.query('TransactionDate >= "2020-03-08"')
merged_data = esso_data_todate.merge(fpcomdb_data_res_todate,how='left',left_on =['PBL','TransactionDate'],\
                right_on = ['PBL','Date'])

In [98]:
merged_data.drop(['GAMADO', 'GAMUL95', 'MRP2ADO',\
                'MRP2UL95', 'ApprovedPriceChangeSumAvgADO',\
                'ApprovedPriceChangeSumAvgUL95',\
                'ApprovedPriceVsMarkerPriceADO', 'ApprovedPriceVsMarkerPriceUL95'],axis=1,inplace=True)

In [101]:
merged_data.to_pickle(r'merged_data.pkl')

In [102]:
merged_data['exclShopSales'] = merged_data['TotalSalesValue']-merged_data['SalesBasketsWithFuel']
merged_data['totalEssoSales'] = merged_data['ApprovedPriceUL95']*merged_data['SalesTransactionVolumeUL95']*0.01+\
                                merged_data['ApprovedPriceADO']*merged_data['SalesTransactionVolumeADO']*0.01
merged_data['shopSalesFromFuel'] = merged_data['SalesBasketsWithFuel']-merged_data['totalEssoSales']

In [113]:
data = pd.read_pickle('df_tesco_marker_V3.pkl')

el_data = pd.read_excel('Elasticity_From_shopSales_ADO_UL_3.xlsx')

In [116]:
el_data.columns

Index(['Site Name', 'PBL', 'const', 'slope', 'rvalu', 'pvalue', 'count',
       'EL_label', 'EL_cat', 'key'],
      dtype='object')

In [102]:
merged_data['exclShopSales'] = merged_data['TotalSalesValue']-merged_data['SalesBasketsWithFuel']
merged_data['exclShopSales'] = merged_data['TotalSalesValue']-merged_data['SalesBasketsWithFuel']
merged_data['exclShopSales'] = merged_data['TotalSalesValue']-merged_data['SalesBasketsWithFuel']

In [122]:
merged_data2 = merged_data.query('exclShopSales >=100 & shopSalesFromFuel >=100')
merged_data2['log_exclShopSales'] =   np.log(merged_data2['exclShopSales'])
merged_data2['log_shopSalesFromFuel'] =   np.log(merged_data2['shopSalesFromFuel'])
merged_data2['logP_by_P0_ADO'] = np.log(merged_data2['ApprovedPriceADO']/merged_data2['MarkerPriceADO'])
merged_data2['logP_by_P0_UL95'] = np.log(merged_data2['ApprovedPriceUL95']/merged_data2['MarkerPriceUL95'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [126]:
el_data_excl_ado = el_data.query("EL_label == 'ExclShopSalesEL_ADO' & key == 'exclShopSales' ")
merged_data3 = merged_data2.merge(el_data_excl_ado[[ 'PBL', 'const', 'slope']],how='left',on=['PBL'])

merged_data3['pred_log_exclShopSales_ADO'] = merged_data3.const + merged_data3.slope*merged_data3.logP_by_P0_ADO
#merged_data3['pred_log_exclShopSales_UL'] = merged_data3.const + merged_data3.slope*merged_data3.logP_by_P0_UL95

merged_data3['mape_exclshopSales_ADO'] = (abs(merged_data3.log_exclShopSales - \
                                merged_data3.pred_log_exclShopSales_ADO)/merged_data3.log_exclShopSales)*100

merged_data3.groupby('SiteName_x')['mape_exclshopSales_ADO'].mean().to_csv('mape_exclshopSales_ADO.csv')

In [138]:
el_data_excl_ul = el_data.query("EL_label == 'ExclShopSalesEl_UL' & key == 'exclShopSales' ")
merged_data4 = merged_data2.merge(el_data_excl_ul[[ 'PBL', 'const', 'slope']],how='left',on=['PBL'])

merged_data4['pred_log_exclShopSales_UL'] = merged_data4.const + merged_data4.slope*merged_data4.logP_by_P0_ADO
merged_data4['mape_exclshopSales_UL'] = (abs(merged_data4.log_exclShopSales - \
                                merged_data4.pred_log_exclShopSales_UL)/merged_data4.log_exclShopSales)*100

merged_data4.groupby('SiteName_x')['mape_exclshopSales_UL'].mean().to_csv('mape_exclshopSales_UL.csv')

In [152]:
el_data_shop_ADO = el_data.query("EL_label == 'ShopSalesFromFuelEl_ADO' ")
merged_data5 = merged_data2.merge(el_data_shop_ADO[[ 'PBL', 'const', 'slope']],how='left',on=['PBL'])

merged_data5['pred_log_shopSalesFromFuel_ADO'] = merged_data5.const + merged_data5.slope*merged_data5.logP_by_P0_ADO
merged_data5['mape_shopSalesFromFuel_ADO'] = (abs(merged_data5.log_shopSalesFromFuel - \
                                merged_data5.pred_log_shopSalesFromFuel_ADO)/merged_data5.log_shopSalesFromFuel)*100

merged_data5.groupby('SiteName_x')['mape_shopSalesFromFuel_ADO'].mean().to_csv('mape_shopSalesFromFuel_ADO.csv')

In [153]:
el_data_shop_ul = el_data.query("EL_label == 'ShopSalesFromFuelEl_UL' ")
merged_data6 = merged_data2.merge(el_data_shop_ul[[ 'PBL', 'const', 'slope']],how='left',on=['PBL'])

merged_data6['pred_log_shopSalesFromFuel_UL'] = merged_data6.const + merged_data5.slope*merged_data6.logP_by_P0_ADO
merged_data6['mape_shopSalesFromFuel_UL'] = (abs(merged_data6.log_shopSalesFromFuel - \
                                merged_data6.pred_log_shopSalesFromFuel_UL)/merged_data6.log_shopSalesFromFuel)*100

merged_data6.groupby('SiteName_x')['mape_shopSalesFromFuel_UL'].mean().to_csv('mape_shopSalesFromFuel_UL.csv')